In [1]:
import json
import pandas as pd
import langid
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import gensim
import cat_detector as det
import cat_detector
from cat_detector import known
import spacy
import re
from val_set import val # validation set

/home/gonzalo/.local/lib/python3.6/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
def get_models(): 
    model_ab = ['es', 'en', 'pt']
    model = {}
    for ab in model_ab:
        model[ab] = gensim.models.KeyedVectors.load_word2vec_format('./wiki.es.align.vec')
    return model

# Language stopwords
def get_stopwords():
    nltk.download('stopwords')
    stop_words = {}
    stop_words["en"] = stopwords.words('english')
    stop_words["es"] = stopwords.words('spanish')
    stop_words["fr"] = stopwords.words('french')
    stop_words["pt"] = stopwords.words('portuguese')
    return stop_words


# Language detection
language_list = ['en','es']
langid.set_languages(language_list)

stop_words = get_stopwords()
model = get_models()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gonzalo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def classify_words(w, stop_words, model, attr, knn):
    clas = cat_detector.get_attr_cat(w.lower(), model[lang], knn, known)
    if clas is not None:
        attr[clas].append(w.lower())

In [4]:
knn = det.load_knn("model_v6.sav")

In [5]:
from googletrans import Translator

In [6]:
count = 0
correct = 0
detected = 0
incorrect = []
for t in val:
    if t[1] != '':
        count += 1
        if "en" == t[1]:
            try:
                translator = Translator()
                w = translator.translate(t[0] , src="en", dest ='es').text
                lang = "es"
            except:
                w = t[0]
                lang = t[1]
        else:
            w = t[0]
            lang = t[1]
        clas = cat_detector.get_attr_cat(w.lower(), model[lang], knn, known)
        print (t[0], t[2], clas)
        if (clas == t[2]):
            correct += 1
        else:
            incorrect.append((t[0], t[1], t[2], clas))
        if clas in ["Forma", "Material","Color", "Estilo", "Habitación", "Mueble"]:
            detected += 1

moderno Estilo Estilo
minimalist Estilo Estilo
basic Estilo not_an_attr
basico Estilo Estilo
small Forma not_an_attr
bedroom Habitación Habitación
basement Habitación Habitación
washroom Habitación Habitación
toilet Habitación Habitación
restroom Habitación not_an_attr
pastel Color not_an_attr
gamer Estilo not_an_attr
beige Color Color
iron Material Material
wood Material Material
acacia Material Material
modern Estilo Estilo
rústico Estilo Estilo
balcony Habitación Habitación
living room Habitación Habitación
Casual Estilo not_an_attr
vintage Estilo Estilo
Grey Color Color
Rustic Estilo Estilo
nordic Estilo Estilo
scandinavian Estilo Estilo
modernos Estilo Estilo
modernas Estilo Estilo
retro Estilo not_an_attr
french Estilo not_an_attr
casual Estilo not_an_attr
salón Habitación Habitación
office Habitación not_an_attr
cromado Color Material
steel Material Material
metal Material Material
cardboard Material Material
leather Material Material
industrial Estilo Estilo
rústicos Estilo Est

In [7]:
count

319

In [8]:
correct/count

0.7366771159874608

In [9]:
detected/count

0.8025078369905956

In [10]:
incorrect

[('basic', 'en', 'Estilo', 'not_an_attr'),
 ('small', 'en', 'Forma', 'not_an_attr'),
 ('restroom', 'en', 'Habitación', 'not_an_attr'),
 ('pastel', 'es', 'Color', 'not_an_attr'),
 ('gamer', 'en', 'Estilo', 'not_an_attr'),
 ('Casual', 'en', 'Estilo', 'not_an_attr'),
 ('retro', 'es', 'Estilo', 'not_an_attr'),
 ('french', 'en', 'Estilo', 'not_an_attr'),
 ('casual', 'en', 'Estilo', 'not_an_attr'),
 ('office', 'en', 'Habitación', 'not_an_attr'),
 ('cromado', 'es', 'Color', 'Material'),
 ('antique', 'en', 'Estilo', 'not_an_attr'),
 ('ergonomic', 'en', 'Forma', 'not_an_attr'),
 ('garden', 'en', 'Habitación', 'Color'),
 ('paper', 'en', 'Material', 'Mueble'),
 ('interior', 'es', 'Habitación', 'Color'),
 ('wide', 'en', 'Forma', 'not_an_attr'),
 ('cabinet', 'en', 'Habitación', 'not_an_attr'),
 ('outdoor', 'en', 'Habitación', 'Color'),
 ('indian', 'en', 'Estilo', 'not_an_attr'),
 ('quarto', 'pt', 'Habitación', 'Material'),
 ('palestinian', 'en', 'Estilo', 'not_an_attr'),
 ('dark', 'en', 'Color', 'n